In [4]:
from azureml.core import Workspace, Dataset
from azureml.core import Experiment

subscription_id = 'd0f7cb58-5c08-4ead-ac4c-29752d3ba0e1'
resource_group = 'mlops-aug-batch'
workspace_name = 'intellipat-mlops'

workspace = Workspace(subscription_id, resource_group, workspace_name)



In [5]:
experiment = Experiment(workspace=workspace, name="iris-experiment")

dataset = Dataset.get_by_name(workspace, name='Iris')
df = dataset.to_pandas_dataframe() #dont forget to edit this line

features = ['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']
taget = 'Species'

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
X_train, X_test, y_train, y_test = train_test_split(df[features],df[taget] , test_size=0.1, shuffle=True)
#step-1: initialise the model class
clf = DecisionTreeClassifier(criterion="entropy") #Information gain as criteria
#step-2: train the model on training set
clf.fit(X_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [9]:
from sklearn.metrics import accuracy_score
import joblib

#step-3 evaluate the data on testing set
y_pred = clf.predict(X_test)
run = experiment.start_logging()
run.log("accuracy",accuracy_score(y_test,y_pred)*100)

model_name = "iris_dt"+ ".pkl"
filename = "outputs/" + model_name
joblib.dump(value=clf, filename=filename)
run.upload_file(name=model_name, path_or_stream=filename)
run.complete()

In [26]:
import sklearn

from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration


model = Model.register(workspace=workspace,
                       model_name='iris',                # Name of the registered model in your workspace.
                       model_path='outputs/iris_dt.pkl',  # Local file to upload and register as a model.
                       model_framework=Model.Framework.SCIKITLEARN,  # Framework used to create the model.
                       model_framework_version=sklearn.__version__,  # Version of scikit-learn used to create the model.
                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=0.5),
                       description='DT classifeier for Iris dataset',
                       tags={'type': 'regression'})

print('Name:', model.name)
print('Version:', model.version)

Registering model iris
Name: iris
Version: 3
